# Load environmental readings

In [1]:
import glob
import math
import pandas as pd
import os

In [2]:
environmental_data = [pd.read_csv(filename, names=["Humidity", "Temperature", "TemperatureF", "Light", "eCO2", "TVOC", "DateTime"]) for filename in glob.glob("./data/environmental-data/*.TXT")]
environment_df = pd.concat(environmental_data, axis=0)

In [3]:
environmental_data

[   Humidity  Temperature  TemperatureF  Light  eCO2  TVOC             DateTime
 0      36.9         21.7         71.06  484.0   415     2  2022-12-12T13:02:04
 1      36.8         21.7         71.06  489.0   641    36  2022-12-12T13:03:04
 2      37.6         21.8         71.24  496.0   426     3  2022-12-12T13:04:04,
    Humidity  Temperature  TemperatureF  Light  eCO2  TVOC             DateTime
 0      37.0         21.8         71.24  495.0   400     0  2022-12-12T13:05:31
 1      36.6         21.7         71.06  493.0   432     4  2022-12-12T13:06:31,
     Humidity  Temperature  TemperatureF   Light  eCO2  TVOC  \
 0       35.1         21.7         71.06   841.0   400     0   
 1       35.0         21.5         70.70   868.0   400     0   
 2       35.0         21.5         70.70  1101.0   400     0   
 3       35.2         21.5         70.70  1085.0   400     0   
 4       35.1         21.5         70.70  1048.0   405     0   
 5       35.1         21.4         70.52  1075.0   411

In [4]:
environment_df

,Humidity,Temperature,TemperatureF,Light,eCO2,TVOC,DateTime
0,36.9,21.7,71.06,484.0,415.0,2.0,2022-12-12T13:02:04
1,36.8,21.7,71.06,489.0,641.0,36.0,2022-12-12T13:03:04
2,37.6,21.8,71.24,496.0,426.0,3.0,2022-12-12T13:04:04
0,37.0,21.8,71.24,495.0,400.0,0.0,2022-12-12T13:05:31
1,36.6,21.7,71.06,493.0,432.0,4.0,2022-12-12T13:06:31
...,...,...,...,...,...,...,...
62,36.1,21.3,70.34,564.0,415.0,2.0,2022-12-12T10:54:39
63,36.3,21.3,70.34,532.0,650.0,38.0,2022-12-12T10:55:39
64,36.6,21.2,70.16,528.0,444.0,6.0,2022-12-12T10:56:39
65,35.9,21.3,70.34,507.0,882.0,73.0,2022-12-12T11:00:06


# Load HRV data

In [5]:
def get_hrv_data_per_user(text_folder):
    #Find the text files
    text_files = []
    for root, folder, files in os.walk(text_folder):
        for file in files:
            if file.endswith('.txt'):
                fullname = os.path.join(root, file)
                text_files.append(fullname)
    text_files.sort() #Sort the filenames

    #Read each of them to a dataframe
    for filenum, file in enumerate(text_files, 1):
        if filenum==1:
            dataframe = pd.read_csv(file, names=['RR'], delim_whitespace=True)
            date = os.path.basename(file).split(".")[0].split(" ")[0]
            time = os.path.basename(file).split(".")[0].split(" ")[1]
            split_date = date.split("-")
            split_time = time.split("-")
            dataframe['DateTime']= split_date[2] + "-" + split_date[1] + "-" + split_date[0] + " " + split_time[0] + ":" + split_time[1] + ":" + split_time[2]
            dataframe['DateTime'] = pd.to_datetime(dataframe['DateTime'])
        else:
            tempdf = pd.read_csv(file, names=['RR'], delim_whitespace=True)
            date = os.path.basename(file).split(".")[0].split(" ")[0]
            time = os.path.basename(file).split(".")[0].split(" ")[1]
            split_date = date.split("-")
            split_time = time.split("-")
            tempdf['DateTime']=split_date[2]+"-"+split_date[1]+"-"+split_date[0]+ " " + split_time[0] + ":" + split_time[1] + ":" + split_time[2]
            tempdf['DateTime'] = pd.to_datetime(tempdf['DateTime'])
            dataframe = pd.concat([dataframe, tempdf], ignore_index=True)

    return dataframe

In [6]:
def calculate_hrv(dataframe):
    array = dataframe[["RR"]].to_numpy()
    sum_of_successive_difference = 0.0
    array = array[(array <= 2000)]
    for idx, x in enumerate(array):
        if x != array[-1] :
            sum_of_successive_difference += (array[idx] - array[idx + 1])**2
            dataframe['RMSSD'] = math.sqrt(sum_of_successive_difference / len(array))
    return dataframe

## Niek

In [7]:
dfHRVNiek = get_hrv_data_per_user(r'./Data/HRV/Niek_Snijders')

In [8]:
dfHRVNiek = dfHRVNiek.groupby(["DateTime"]).apply(lambda x: calculate_hrv(x))

In [9]:
dfHRVNiek["User Name"] = "niek@email.com"

In [10]:
dfHRVNiek.head()

,RR,DateTime,RMSSD,User Name
0,776,2022-11-17 15:17:06,42.056698,niek@email.com
1,758,2022-11-17 15:17:06,42.056698,niek@email.com
2,737,2022-11-17 15:17:06,42.056698,niek@email.com
3,711,2022-11-17 15:17:06,42.056698,niek@email.com
4,738,2022-11-17 15:17:06,42.056698,niek@email.com


In [11]:
dfHRVNiek = dfHRVNiek.drop(["RR"], axis = 1)

In [12]:
dfHRVNiek.drop_duplicates(keep='first', inplace=True)

In [13]:
dfHRVNiek.reset_index(inplace=True)
dfHRVNiek = dfHRVNiek.drop(["index"], axis = 1)

In [14]:
dfHRVNiek.head(n=100)

,DateTime,RMSSD,User Name
0,2022-11-17 15:17:06,42.056698,niek@email.com
1,2022-11-18 11:00:00,42.440603,niek@email.com
2,2022-11-24 12:01:05,45.918517,niek@email.com
3,2022-11-30 11:07:17,36.091961,niek@email.com
4,2022-11-30 11:49:08,52.136928,niek@email.com
...,...,...,...
70,2022-12-19 12:17:49,50.244645,niek@email.com
71,2022-12-19 12:28:11,61.783493,niek@email.com
72,2022-12-19 12:38:01,100.819244,niek@email.com
73,2022-12-19 12:48:43,71.786608,niek@email.com


## Job

In [15]:
dfHRVJob = get_hrv_data_per_user(r'./Data/HRV/jobhaast@hotmail.com')

In [16]:
dfHRVJob = dfHRVJob.groupby(["DateTime"]).apply(lambda x: calculate_hrv(x))

In [17]:
dfHRVJob["User Name"] = "job@email.com"

In [18]:
dfHRVJob = dfHRVJob.drop(["RR"], axis = 1)

In [19]:
dfHRVJob.drop_duplicates(keep='first', inplace=True)

In [20]:
dfHRVJob.reset_index(inplace=True)
dfHRVJob = dfHRVJob.drop(["index"], axis = 1)

In [21]:
dfHRVJob.head(n=100)

,DateTime,RMSSD,User Name
0,2022-06-12 14:38:47,57.882491,job@email.com
1,2022-06-12 14:41:41,82.642453,job@email.com
2,2022-06-12 14:49:55,129.602890,job@email.com
3,2022-07-12 10:30:46,58.791368,job@email.com
4,2022-07-12 13:11:45,67.209265,job@email.com
...,...,...,...
95,2022-12-19 13:09:21,63.746860,job@email.com
96,2022-12-19 13:19:22,151.740676,job@email.com
97,2022-12-19 13:29:18,47.881883,job@email.com
98,2022-12-19 13:39:18,104.573140,job@email.com


## Stefan

In [22]:
dfHRVStefan = get_hrv_data_per_user(r'./Data/HRV/s.jaspers1997@gmail.com')

In [23]:
dfHRVStefan = dfHRVStefan.groupby(["DateTime"]).apply(lambda x: calculate_hrv(x))

In [24]:
dfHRVStefan["User Name"] = "stefan@jaspers.nl"

In [25]:
dfHRVStefan = dfHRVStefan.drop(["RR"], axis = 1)

In [26]:
dfHRVStefan.drop_duplicates(keep='first', inplace=True)

In [27]:
dfHRVStefan.reset_index(inplace=True)
dfHRVStefan = dfHRVStefan.drop(["index"], axis = 1)

In [28]:
dfHRVStefan.head(n=100)

,DateTime,RMSSD,User Name
0,2022-07-12 10:25:25,42.801957,stefan@jaspers.nl
1,2022-07-12 13:54:44,61.043251,stefan@jaspers.nl
2,2022-07-12 13:59:15,120.665735,stefan@jaspers.nl
3,2022-12-12 13:24:39,48.239248,stefan@jaspers.nl
4,2022-12-12 13:52:36,56.135213,stefan@jaspers.nl
5,2022-12-12 14:34:22,74.210511,stefan@jaspers.nl
6,2022-12-12 14:47:10,65.346911,stefan@jaspers.nl
7,2022-12-14 11:27:23,154.013215,stefan@jaspers.nl
8,2022-12-14 11:53:47,27.794275,stefan@jaspers.nl
9,2022-12-14 12:10:59,39.124237,stefan@jaspers.nl


## Noah

In [29]:
dfHRVNoah = get_hrv_data_per_user(r'./Data/HRV/Noah')

In [30]:
dfHRVNoah = dfHRVNoah.groupby(["DateTime"]).apply(lambda x: calculate_hrv(x))

In [31]:
dfHRVNoah["User Name"] = "Noah@email.com"

In [32]:
dfHRVNoah = dfHRVNoah.drop(["RR"], axis=1)

In [33]:
dfHRVNoah.drop_duplicates(keep='first', inplace=True)

In [34]:
dfHRVNoah.reset_index(inplace=True)
dfHRVNoah = dfHRVNoah.drop(["index"], axis=1)

In [35]:
dfHRVNoah.head(n=100)

,DateTime,RMSSD,User Name
0,2022-07-12 14:11:10,53.089728,Noah@email.com
1,2022-07-12 14:14:30,174.320772,Noah@email.com
2,2022-07-12 14:18:06,127.283935,Noah@email.com
3,2022-07-12 14:19:29,142.566617,Noah@email.com
4,2022-12-12 13:28:19,30.287191,Noah@email.com
...,...,...,...
95,2022-12-20 13:44:27,28.399407,Noah@email.com
96,2022-12-20 13:48:54,94.160586,Noah@email.com
97,2022-12-20 13:59:06,278.772389,Noah@email.com
98,2022-12-20 14:01:14,118.162658,Noah@email.com


# Load self reporting data

In [36]:
import pandas as pd
environmental_data = [pd.read_csv(filename) for filename in glob.glob("./data/selfreporting/*.csv")]
self_reporting_df = pd.concat(environmental_data, axis=0)

In [37]:
self_reporting_df.head(n=100)

,Air Qualityall Good,Beverage,Cloth 1,Cloth 2,Cloth 3,Cloth 4,Cloth 5,Cloth 6,Cloth 7,Duration Of Location,...,Location,Mode Of Transport,Mood,Smelly,Stuffy,Suffocating,Thermal Comfort,Thermal Preference,Timestamp,User Name
0,YES,2,NO,YES,YES,NO,NO,NO,NO,1-2 hours,...,LA Explora floor 2,1,3,NO,NO,NO,5,1,2022/12/7 2022-12-07 10:42:04.299,job@email.com
1,YES,1,NO,YES,YES,NO,NO,NO,NO,more than 3 hours,...,LA402,1,3,NO,NO,NO,4,2,2022/12/7 2022-12-07 13:51:00.415,job@email.com
2,YES,2,NO,YES,YES,NO,NO,NO,NO,more than 3 hours,...,LA402,1,4,NO,NO,NO,5,2,2022/12/7 2022-12-07 14:04:39.307,job@email.com
3,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA222,1,4,NO,NO,NO,5,1,2022/12/12 2022-12-12 13:34:15.607,job@email.com
4,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,NO,NO,5,1,2022/12/12 2022-12-12 13:42:56.603,job@email.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,YES,2,NO,YES,YES,YES,NO,NO,NO,more than 3 hours,...,LD124,1,5,NO,NO,NO,3,2,2022/12/19 2022-12-19 13:39:37.144,job@email.com
96,YES,2,NO,YES,YES,YES,NO,NO,NO,more than 3 hours,...,LD124,1,4,NO,NO,NO,3,2,2022/12/19 2022-12-19 13:49:39.995,job@email.com
97,YES,2,NO,YES,YES,YES,NO,NO,NO,more than 3 hours,...,LD124,1,4,NO,NO,NO,2,2,2022/12/19 2022-12-19 13:59:44.919,job@email.com
98,YES,2,NO,YES,YES,YES,NO,NO,NO,more than 3 hours,...,LD124,1,4,NO,NO,NO,3,2,2022/12/19 2022-12-19 14:09:53.998,job@email.com


## Self reporting data preprocessing

In [38]:
def bool_to_num(val):
    if val.lower() == "yes": return 1
    return 0

columns = ["Air Qualityall Good", "Smelly", "Stuffy", "Suffocating", "Humid"]
for i in columns:
    self_reporting_df[i] = self_reporting_df[i].apply(lambda x: bool_to_num(x))

for i in range(1, 8):
    column = f"Cloth {i}"
    self_reporting_df[column] = self_reporting_df[column].apply(lambda x: bool_to_num(x))

dummies = ["Duration Of Location", "Eat Recent", "Location"]
self_reporting_df = pd.get_dummies(self_reporting_df, columns=dummies)
self_reporting_df

,Air Qualityall Good,Beverage,Cloth 1,Cloth 2,Cloth 3,Cloth 4,Cloth 5,Cloth 6,Cloth 7,Humid,...,Location_LA222,Location_LA223,Location_LA402,Location_LD022,Location_LD025,Location_LD027,Location_LD106,Location_LD124,Location_LD323,Location_LD328
0,1,2,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,2,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,2,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,1,2,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,0,2,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
20,0,2,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
21,0,2,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
22,1,2,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


# Date time formatting

In [39]:
environment_df['DateTime'] = pd.to_datetime(environment_df['DateTime'])

In [40]:
datetime = self_reporting_df['Timestamp'].str.split(".", n = 1, expand = True)
self_reporting_df['Timestamp'] = datetime[0]
datetime = self_reporting_df['Timestamp'].str.split(" ", n = 1, expand = True)
datetime
self_reporting_df['Timestamp'] = datetime[1]

In [41]:
self_reporting_df['DateTime'] = pd.to_datetime(self_reporting_df['Timestamp'])

# Merge data

In [42]:
# Merge hrv dataframes
dfHRV = pd.concat([dfHRVNiek, dfHRVJob, dfHRVStefan, dfHRVNoah], axis=0)
dfHRV.head(n=100)

,DateTime,RMSSD,User Name
0,2022-11-17 15:17:06,42.056698,niek@email.com
1,2022-11-18 11:00:00,42.440603,niek@email.com
2,2022-11-24 12:01:05,45.918517,niek@email.com
3,2022-11-30 11:07:17,36.091961,niek@email.com
4,2022-11-30 11:49:08,52.136928,niek@email.com
...,...,...,...
20,2022-12-13 11:26:12,41.213666,job@email.com
21,2022-12-13 11:37:19,47.644108,job@email.com
22,2022-12-13 11:46:08,84.984259,job@email.com
23,2022-12-13 12:27:19,51.829427,job@email.com


In [43]:
new_df = pd.merge_asof(dfHRV.sort_values('DateTime'), self_reporting_df.sort_values('DateTime'), on='DateTime', by='User Name', tolerance=pd.Timedelta('420s'), direction='nearest')

In [44]:
# Merge df for readings with self reporting df on DateTime and get the nearest
df = pd.merge_asof(new_df.sort_values('DateTime'), environment_df.sort_values('DateTime'), on='DateTime', tolerance=pd.Timedelta('120s'), direction='nearest')

In [45]:
new_df

,DateTime,RMSSD,User Name,Air Qualityall Good,Beverage,Cloth 1,Cloth 2,Cloth 3,Cloth 4,Cloth 5,...,Location_LA222,Location_LA223,Location_LA402,Location_LD022,Location_LD025,Location_LD027,Location_LD106,Location_LD124,Location_LD323,Location_LD328
0,2022-01-12 14:35:12,92.179360,niek@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-12 14:38:47,57.882491,job@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-12 14:41:41,82.642453,job@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-12 14:49:55,129.602890,job@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-12 15:52:08,63.953953,niek@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,2022-12-21 10:23:49,36.055991,job@email.com,1.0,2.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
327,2022-12-21 10:33:23,44.180512,job@email.com,1.0,2.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
328,2022-12-21 10:40:49,120.945170,Noah@email.com,1.0,2.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
329,2022-12-21 10:43:31,31.688050,job@email.com,1.0,2.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [46]:
df

,DateTime,RMSSD,User Name,Air Qualityall Good,Beverage,Cloth 1,Cloth 2,Cloth 3,Cloth 4,Cloth 5,...,Location_LD106,Location_LD124,Location_LD323,Location_LD328,Humidity,Temperature,TemperatureF,Light,eCO2,TVOC
0,2022-01-12 14:35:12,92.179360,niek@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-12 14:38:47,57.882491,job@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-12 14:41:41,82.642453,job@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-12 14:49:55,129.602890,job@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-12 15:52:08,63.953953,niek@email.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,2022-12-21 10:23:49,36.055991,job@email.com,1.0,2.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
327,2022-12-21 10:33:23,44.180512,job@email.com,1.0,2.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
328,2022-12-21 10:40:49,120.945170,Noah@email.com,1.0,2.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
329,2022-12-21 10:43:31,31.688050,job@email.com,1.0,2.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df.dropna(inplace=True)
df.to_csv('final.csv', index=False)